<a href="https://colab.research.google.com/github/E-pep/ThesisLitter1920/blob/master/SplitMobilenetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.tensorflow.org/tutorials/images/transfer_learning


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly
  !pip install tensorflow==2.0.0
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2

In [0]:
print(tf.__version__)

In [0]:
def getLayerIndexByName(model, layername):
    for idx, layer in enumerate(model.layers):
        if layer.name == layername:
            return idx

# **Inladen voorgetraind model**

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Insert path to model here
reloaded = tf.keras.models.load_model("path to model here")
tf.keras.utils.plot_model(reloaded, to_file="path to model here")

In [0]:
reloaded.summary()

# **Find all Add Layers inputs**

In [0]:
def SplitModel(FullModel, SplitLayer):

    #initialize lists
  IndexAddInputsList = []
  IndexAddList = []
  IndexInputsList = []

  #run over model to find all add layers index
  for index in range (0,len(FullModel.layers)):
    if isinstance(FullModel.layers[index], tf.keras.layers.Add):

      #add index of add layer to list 
      tempIndex1 = getLayerIndexByName(FullModel,FullModel.layers[index].input[0].name[:-11])
      tempIndex2 = getLayerIndexByName(FullModel,FullModel.layers[index].input[1].name[:-11])
      IndexAddInputsList.append([index,tempIndex1,tempIndex2, FullModel.layers[index].name])
      IndexAddList.append(index)
      IndexInputsList.append(tempIndex1)
      IndexInputsList.append(tempIndex2)  
      # InputsOfAddLayerList.append(FullModel.layers[index].)
  # AddLayerIndexlist
  IndexAddInputsList
  # IndexInputsList




  FirstModel = tf.keras.Model(inputs=FullModel.input,
                                 outputs=FullModel.get_layer(SplitLayer).output)
  
  idx = getLayerIndexByName(FullModel, SplitLayer)
  idx = idx+1
  input_shape = FullModel.layers[idx].get_input_shape_at(0) # get the input shape of desired layer
  # print(input_shape[1:])
  layer_input = tf.keras.layers.Input(shape=input_shape[1:]) # a new input tensor to be able to feed the desired layer

  # create the new nodes for each layer in the path
  x = layer_input
  teller = idx

  # Variable keeps in mind to switch between x1 and x2
  InputVarSwitch = 0

  # Variable if regular after a Inputlayer or Add layer
  #0 normal
  #1 After Add layer
  #2 After Input layer
  RegularAfterSpecial = 0


  for layer in FullModel.layers[idx:]:

    #check if index of Input of Add is reached:
    if teller in IndexAddList:
      print ("Add:",teller, IndexX1, IndexX2)
      Addlayer = tf.keras.layers.Add()([x1, x2])
      RegularAfterSpecial = 1
    #check if layer is input of Adder

    if teller in IndexInputsList:

      
      if InputVarSwitch == 0:
        if RegularAfterSpecial == 1:
          print("kom ik hier1?")
          x1 = Addlayer
        else:
          x1 = layer(x)
          print ("input x1:",teller)
        RegularAfterSpecial = 2
        InputVarSwitch = 1
        IndexX1 = teller
      else:
        if RegularAfterSpecial == 1:
          print("kom ik hier2?")
          x2 = Addlayer
        else:
          x2 = layer(x)
          print ("input x2:",teller)
          print("shape:",x2.shape)
        RegularAfterSpecial = 2
        InputVarSwitch = 0
        IndexX2 = teller
      
    elif teller in IndexAddList:
      print("verdergaan")
      q = 1+1

    # if we have a regular layer
    else:
      print("RegularAfterSpecial:",RegularAfterSpecial)
      if RegularAfterSpecial == 0:
        x = layer(x)
        print("regular layer:", teller)
      elif RegularAfterSpecial == 1:
        print("shape:",Addlayer.shape, "layer:", teller)
        x = layer(Addlayer)
        print("regular layer after add:", teller, layer.__class__.__name__)
      elif RegularAfterSpecial == 2:
        print("regular layer after input:", teller)
        
        if InputVarSwitch ==1:
          x = layer(x1)
        else:
          x = layer(x2)
      RegularAfterSpecial = 0
      # x = layer(x)

    # if isinstance(layer, tf.keras.layers.Add):   
      #find layer in model
      # p = layer.input[0].name
      # tf.keras.layers.Add()([x1, x2])
      # x = layer(x)
    teller += 1





  # create the model
  SecondModel = tf.keras.Model(layer_input, x)
  return FirstModel, SecondModel

In [0]:
# path where to save 
def SaveModel(FirstModel, SecondModel, modelIdx):
  export_path = "path where to save/model{}/first".format(int(modelIdx))
  os.makedirs(export_path)
  export_path += "/FirstModel"
  tf.keras.models.save_model(FirstModel, export_path, save_format='h5')
  tf.keras.utils.plot_model(FirstModel, to_file='path where to save/model{}/Firstmodel.png'.format(int(modelIdx)))


  export_path = "path where to save/model{}/second".format(int(modelIdx))
  os.makedirs(export_path)
  export_path += "/SecondModel"
  tf.keras.models.save_model(SecondModel, export_path, save_format='h5')
  tf.keras.utils.plot_model(SecondModel, to_file='path where to save/model{}/Secondmodel.png'.format(int(modelIdx)))

In [0]:
LayersToSplitList = ['expanded_conv_project_BN', 'block_1_expand_relu', 'block_1_project', 'block_2_add', 'block_3_expand_relu', 'block_3_project', 'block_5_add'
                      ,'block_6_expand_relu', 'block_6_project',  'block_9_add', 'block_10_project', 'block_12_add', 'block_13_expand_relu', 
                     'block_15_add', 'block_16_project', 'global_average_pooling2d', 'dense']

teller = 1
for layer in LayersToSplitList:
  FirstModel, SecondModel = SplitModel(reloaded, layer)
  SaveModel(FirstModel, SecondModel, teller)
  teller+=1



FirstModel.summary()